### Compute and store filtered and demodulated velocity fields at each grid point

In [1]:
import numpy as np
import geopandas as gpd

import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im

from sympy import Symbol, pi, atan, factor, lambdify

import mitequinox.plot as pl
import mitequinox.sigp as sp

from fsspec.implementations.local import LocalFileSystem

INFO: Compiled ParcelsRandom ==> /dev/shm/pbs.8762128.datarmor0/parcels-502090/libparcels_random_b5c1b2c1-6458-4d0a-a9cb-71d0a5e8cd94.so


In [36]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster() #processes=7, cores=7
w = cluster.scale(jobs=12
                 )
client = Client(cluster)

In [38]:
client

Client Scheduler: tcp://10.148.0.105:42137 Dashboard: http://10.148.0.105:8787/status,Cluster Workers: 84 Cores: 84 Memory: 1.31 TiB


In [39]:
def convolve(x, h=None, hilbert=False):
    """ Convolve an input signal with a kernel
    Optionaly compute the Hilbert transform of the resulting time series
    
    Parameters
    x : input signal
    h : filter 
    hilbert : True for Hilbert transform to be applied to the filtered signal
    
    Returns
    x_f : filtered signal or hilbert transform of the filtered signal
    """
    x_f = signal.filtfilt(h, [1], x, axis=-1,padlen=0)#
    if hilbert:
        return signal.hilbert(x_f)
    else:
        return x_f

def filt(v, h, hilbert=False):
    
    output_dtype = complex if hilbert else float
    gufunc_kwargs = dict(output_sizes={'time': len(v.time)})
    return xr.apply_ufunc(convolve, v, kwargs={'h': h, 'hilbert': hilbert},
                    dask='parallelized', output_dtypes=[output_dtype],
                    input_core_dims=[['time']],
                    output_core_dims=[['time']],
                    dask_gufunc_kwargs = gufunc_kwargs,
                         )

##### First step : Filter and demodulate for each face

In [40]:
# Load Eulerian fields
ds = xr.open_zarr(ut.work_data_dir+'rechunked/SSV_rot.zarr')
grd = ut.load_grd()[['XC', 'YC', 'Depth']]#.persist()

In [41]:
isel=dict(face=6) #face
V = ['SSV_rot']#'zonal_velocity','meridional_velocity'
path = '/home1/datawork/zcaspar/mit4320/filtered_itide/'
dsel = 100

In [42]:
dt = 1/24 # time step in days

tidal_omega = sp.get_tidal_frequencies("M2", "K2","S2","N2")
omega_M2,omega_S2,omega_N2,omega_K2, domega, name = tidal_omega["M2"],tidal_omega["S2"],tidal_omega["N2"],tidal_omega["K2"], .2, "semidiurnal"
omega = (omega_M2+omega_S2)/2#center frequency
Tw = 30 #filter length
dband = 0.2 # half-bandwidth
V = ['SSV_rot']

In [43]:
import dask
def wrap_filter(_ds,dt=dt):
    h = sp.generate_filter(om, T=Tw, dt=dt, bandwidth=dband, normalized_bandwidth=None)
    time = np.arange(0,_ds['time'].values.size*dt,dt)
    exp = np.exp(-1j*om*2*np.pi*time)
#    _ds = _ds.chunk(dict(time=16,i=100,j=100))
    ds_hat={}
    for v in V:
        ds_hat[v+'_hat'] = filt(_ds[v], h,hilbert=True)#.persist()
    #        ds_hat[v+'_hat'] = ds_hat[v+'_hat'].assign_coords({'lon':ds_hat[v+'_hat'].lon,'lat':ds_hat[v+'_hat'].lat})
        ds_hat[v+'_demodulated'] = ds_hat[v+'_hat']*exp
            #ds_hat[v+'_demodulated_real']=ds_hat[v+'_demodulated'].real
            #ds_hat[v+'_demodulated_imag']=ds_hat[v+'_demodulated'].imag
    ds_hat = xr.merge([ds_hat[v].rename(v) for v in ds_hat.keys()])
#    ds_hat = ds_hat.chunk(dict(time=16,i=4320//4))
#    ds_hat = ut._reset_chunk_encoding(ds_hat)
    return ds_hat

In [44]:
#Select and rechunk (empirical) original dataset
dsp = ds.isel(j=slice(0,2300))#.isel(**isel)#.sel(**sel)#.persist()
dsp = dsp.chunk({"time": -1})#.persist()#,'i':100,'j':70

In [45]:
om,Tw,dband,V = omega,Tw,dband,V #define parameters needed in wrap_filter

In [46]:
# Apply wrap_filter on the dataset and store the result for each face (seperated)
import os
#zarr_main = os.path.join(ut.root_data_dir, "filtered_itide/SSU_filtered_face4.zarr")
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_out, zarr = ut.custom_distribute(dsp, 
                                        wrap_filter,
                                        overwrite=True,
                                        suffix="SSV_filtered_2300.zarr",
                                        tmp_dir=os.path.join(ut.root_data_dir, "filtered_itide"),
                                        append=True,
                                        face=1
#                                        j=dsel
                                       )

0it [00:00, ?it/s]distributed.utils - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 321, in update
    self.root.title.text = title
AttributeError: 'str' object has no attribute 'text'
distributed.utils - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 2263, in status_doc
    nbytes_cluster.update()
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/bok

KeyboardInterrupt: 

In [38]:
ds_out

,Array,Chunk
Bytes,2.35 TiB,26.37 MiB
Shape,"(4320, 4320, 8640)","(100, 1080, 16)"
Count,95041 Tasks,95040 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,2.35 TiB,26.37 MiB
Shape,"(4320, 4320, 8640)","(100, 1080, 16)"
Count,95041 Tasks,95040 Chunks
Type,complex128,numpy.ndarray


In [47]:
#client.restart()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


##### Second step : Concat filtered and demodulated fields for each face

In [18]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()

#
from dask_jobqueue import PBSCluster
cluster = PBSCluster(walltime='04:00:00') #processes=7, cores=7 ,walltime='03:00:00'
w = cluster.scale(jobs=10
                 )
client = Client(cluster)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [20]:
client

Client Scheduler: tcp://10.148.0.105:53471 Dashboard: http://10.148.0.105:8787/status,Cluster Workers: 70 Cores: 70 Memory: 1.09 TiB


In [21]:
path = '/home/datawork-lops-osi/equinox/mit4320/filtered_itide/'

#ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered_face%s.zarr'%i
#                           ) for i in np.arange(11,12)],dim='face')#.persist()
ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered_%s.zarr'%i
                           ) for i in np.arange(0,5)],dim='face')
#ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i
#                           ) for i in ['00','01']],dim='face')

In [22]:
ds

,Array,Chunk
Bytes,28.16 TiB,18.46 MiB
Shape,"(12, 4320, 4320, 8640)","(1, 70, 1080, 16)"
Count,3214085 Tasks,1607040 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,28.16 TiB,18.46 MiB
Shape,"(12, 4320, 4320, 8640)","(1, 70, 1080, 16)"
Count,3214085 Tasks,1607040 Chunks
Type,complex128,numpy.ndarray


In [23]:
def concat(ds):
#    _ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered_%s.zarr'%i
#                           ) for i in np.arange(0,5)],dim='face')
#    _ds = ds#xr.concat(ds,dim='face')
    _ds = ds.chunk(dict(time=16,i=4320//4
                        )) #,j=4320//4
#    _ds = ut._reset_chunk_encoding(_ds)
    return _ds

In [6]:
ds = ds.persist()

In [26]:
#Apply concat function on ds
face = 1
import os, dask
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_out, zarr = ut.custom_distribute(ds, 
                                        concat,
                                        overwrite=True,
                                        suffix="SSV_filtered.zarr",
                                        tmp_dir=os.path.join(ut.root_data_dir, "filtered_itide"),
                                        append=True,
#                                        face=face
                                        j=200
                                       )

0it [00:00, ?it/s]distributed.utils - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 321, in update
    self.root.title.text = title
AttributeError: 'str' object has no attribute 'text'
distributed.utils - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 2263, in status_doc
    nbytes_cluster.update()
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/bok

KeyboardInterrupt: 

In [7]:
ds_out

,Array,Chunk
Bytes,7.04 TiB,18.46 MiB
Shape,"(3, 4320, 4320, 8640)","(1, 70, 1080, 16)"
Count,401761 Tasks,401760 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,7.04 TiB,18.46 MiB
Shape,"(3, 4320, 4320, 8640)","(1, 70, 1080, 16)"
Count,401761 Tasks,401760 Chunks
Type,complex128,numpy.ndarray


In [27]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
